In [1]:
import pickle
def mkPikle(filename, content):
    with open(filename, 'wb') as f:
        pickle.dump(content, f)
        f.close()
        
def rdPickle(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

Task 1: exclude stopwords. Identify **relative** frequency of lexical words before and after lockdown (data before 2020 and after 2020).

e.g. 1. the lexical word virus (tokens: viruses, virus, etc.) is used 20% per 10000 Tweets before lockdown and 30% after lockdown.<br>
e.g.2 mask is used is used 10% per 10000 Tweets before lockdown and 30% after lockdown.<br>
e.g.3 covid is used 0% before and 20 % after per 1000 Tweets <br>
e.g. 4 distance is used 20% per 10000 Tweets before lockdown and 30% after lockdown.<br>

Theoretical implication: the higher the change in frequency, the more probable is that a word arises in a new topic and/or has a new language use. 

Run n-gram or collocation analysis of the words with the highest frequency difference

Prediction: If a word appears with a new meaning, it will show differences in n-gram/collocations.

e.g. distance will appear with social as in social distancing after but not before the lockdown. 


In [2]:
import pandas as pd
#pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', 200)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

In [3]:
lstTexts        = rdPickle('pickle/2021-4/lstTexts.pkl')
df_lock         = rdPickle('pickle/2021-4/df_lock.pkl')
countVector     = rdPickle('pickle/2021-4/countVector.pkl')
countVectorizer = rdPickle('pickle/2021-4/countVectorizer.pkl')
df_lock_summary = rdPickle('pickle/2021-4/df_lock_summary.pkl')
dicCV_lock      = rdPickle('pickle/2021-4/dicCV_lock.pkl')

lstTextToken        = rdPickle('pickle/2021-4/lstTextToken.pkl')
lstTextToken_org    = rdPickle('pickle/2021-4/lstTextToken_org.pkl')

#count_vect_df   = rdPickle('pickle/2021-4/count_vect_df.pkl')
lstCleanTxt2        = rdPickle('pickle/2021-4/lstCleanTxt2.pkl')

In [5]:
from bertopic import BERTopic
# load Bertopic
modelT = BERTopic.load("pickle/2021-4/modelBertTopic")
dicTopWords  = rdPickle('pickle/2021-4/dicTopWords.pkl')
setTopWords  = rdPickle('pickle/2021-4/setTopWords.pkl')
dfTp         = rdPickle('pickle/2021-4/dfTp.pkl')
topics       = rdPickle('pickle/2021-4/topics.pkl', )
probabilities= rdPickle('pickle/2021-4/probabilities.pkl', )

In [4]:
from gensim.models import Word2Vec
# load model
model = Word2Vec.load('pickle/2021-4/model.bin')
print(model)

Word2Vec(vocab=87968, size=100, alpha=0.025)


Step 1 :Read the file

In [5]:
import pandas as pd
import os
df_lock = pd.read_json(open( "/home/dell/workCode/Phd_works/topic_modeling/Covid_paper_work/data/after_lockdown/combined.json", 
                            "r", encoding="utf8"), lines=True)

In [9]:
df_lock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76008 entries, 0 to 76007
Data columns (total 36 columns):
 #   Column                     Non-Null Count  Dtype              
---  ------                     --------------  -----              
 0   created_at                 76008 non-null  datetime64[ns, UTC]
 1   id                         76008 non-null  int64              
 2   id_str                     76008 non-null  int64              
 3   text                       76008 non-null  object             
 4   source                     76008 non-null  object             
 5   truncated                  76008 non-null  bool               
 6   in_reply_to_status_id      1004 non-null   float64            
 7   in_reply_to_status_id_str  1004 non-null   float64            
 8   in_reply_to_user_id        1477 non-null   float64            
 9   in_reply_to_user_id_str    1477 non-null   float64            
 10  in_reply_to_screen_name    1477 non-null   object             
 11  us

In [10]:
#df_2021.groupby
df_lock.groupby(['lang']).agg({"text": ['count']})

,text
,count
lang,
en,76008


In [8]:
df_lock.groupby(df_lock.timestamp_ms.dt.date).agg({"text": ['count']})

,text
,count
timestamp_ms,
2020-02-02,5008
2020-04-01,3524
2020-04-02,3477
2020-04-03,3602
2020-04-04,2999
2020-04-05,3363
2020-04-06,1932
2020-04-07,3701


In [15]:
df_lock.timestamp_ms.dt.date.value_counts()

2020-02-02    5008
2020-04-07    3701
2020-04-12    3645
2020-04-03    3602
2020-04-10    3556
2020-04-01    3524
2020-04-02    3477
2020-04-15    3394
2020-04-05    3363
2020-04-22    3347
2020-04-18    3323
2020-04-17    3251
2020-04-20    3217
2020-04-11    3205
2020-04-09    3195
2020-04-23    3192
2020-04-21    3127
2020-04-13    3092
2020-04-14    3078
2020-04-04    2999
2020-04-19    2977
2020-04-08    2656
2020-04-06    1932
2020-04-16     147
Name: timestamp_ms, dtype: int64

In [27]:
df_lock['place'].apply(pd.Series).country.value_counts()[:10]

United States                  56101
Canada                          4354
United Kingdom                  2790
Australia                       1426
Nigeria                          785
Republic of the Philippines      577
Indonesia                        501
India                            447
New Zealand                      413
Brasil                           404
Name: country, dtype: int64

In [10]:
df_lock.head()

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,filter_level,lang,timestamp_ms,display_text_range,extended_entities,extended_tweet,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink
0,2020-02-02 23:00:00+00:00,1224105213788852227,1224105213788852224,"Win, lose, or tie, Steelers fan till I die 💛🖤🏈 @ Monaghan's Sports Pub &amp; Grill https://t.co/ijw6AsCG13","<a href=""http://instagram.com"" rel=""nofollow"">Instagram</a>",False,NaN,NaN,NaN,NaN,...,low,en,2020-02-02 23:00:00.399,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-02 23:00:00+00:00,1224105216276094977,1224105216276094976,"Wind 10.1 mph WNW. Barometer 1004.27 hPa, Rising slowly. Temperature 8.8 °C. Rain today 0.2mm. Humidity 83%","<a href=""http://sandaysoft.com/"" rel=""nofollow"">Sandaysoft Cumulus</a>",False,NaN,NaN,NaN,NaN,...,low,en,2020-02-02 23:00:00.992,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-02 23:00:01+00:00,1224105217999822848,1224105217999822848,"Wind 7.9 mph W. Barometer 29.95 in, Steady. Temperature 48.4 °F. Rain today 0.00 in. Humidity 15%","<a href=""http://sandaysoft.com/"" rel=""nofollow"">Sandaysoft Cumulus</a>",False,NaN,NaN,NaN,NaN,...,low,en,2020-02-02 23:00:01.403,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-02 23:00:01+00:00,1224105219954368512,1224105219954368512,#Carpool #Tumwater -&gt; #CrystalMountain rideshare 1 Seat offered #splitcosts #taketurns https://t.co/J6WZVIpvEL,"<a href=""https://www.carpoolworld.com"" rel=""nofollow"">CarpoolWorld Feed</a>",False,NaN,NaN,NaN,NaN,...,low,en,2020-02-02 23:00:01.869,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-02 23:00:01+00:00,1224105220176850944,1224105220176850944,"22:48 Temp. 10°C, Hum. 83%, Dewp. 6.4°C, Bar. 1010.7 hpa, Rain Today 6.4000 mm","<a href=""http://plus.meteoware.com/netatmo/"" rel=""nofollow"">MeteoWare Plus+</a>",False,NaN,NaN,NaN,NaN,...,low,en,2020-02-02 23:00:01.922,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Pre-processing text data

Most of the text data are cleaned by following below steps.

    Remove punctuations
    Tokenization - Converting a sentence into list of words
    Remove stopwords
    Lammetization/stemming - Tranforming any form of a word to its root word



Another Cleanup which removes web links

In [8]:
import re, string, unicodedata
import nltk
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

def replace_contractions(text):
    # creating an empty list
    expanded_words = []    
    for word in text.split():
      # using contractions.fix to expand the shotened words
      expanded_words.append(contractions.fix(word))   
    expanded_text = ' '.join(expanded_words)
    """Replace contractions in string of text"""
    return expanded_text

def remove_URL(sample):
    """Remove URLs from a sample string"""
    return re.sub(r"http\S+", "", sample)

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words


def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

# function to remove special characters
def remove_special_characters(text):
    # define the pattern to keep
    pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 
    return re.sub(pat, '', text)

# function to remove special characters
def remove_extra_whitespace_tabs(text):
    #pattern = r'^\s+$|\s+$'
    pattern = r'^\s*|\s\s*'
    return re.sub(pattern, ' ', text).strip()

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    
    #words = replace_numbers(words)
    words = remove_stopwords(words)
    return words

def normalize1(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    #words = replace_numbers(words)
    #words = remove_stopwords(words)
    return words

def preprocess(sample):
    sample = remove_URL(sample)
    sample = replace_contractions(sample)
    # Tokenize
    words = nltk.word_tokenize(sample)

    # Normalize
    return normalize(words)


In [10]:

def clean_text_v2(text):
    text = remove_URL(text)
    text = replace_contractions(text)
    text = re.sub('[0-9]+', '', text) # remove number
    
    # Tokenize
    tokens = nltk.word_tokenize(text)
    # Normalize
    words = normalize(tokens)

    return words

In [5]:
# Keeping the original text structure
def clean_text_v1(text):
    text = remove_URL(text)
    text = replace_contractions(text)
    text = re.sub('[0-9]+', '', text) # remove number
    text = remove_special_characters(text) # Special char
    text = remove_extra_whitespace_tabs(text) # Extra whitespace
    
    # Tokenize
    tokens = nltk.word_tokenize(text)
    # Normalize
    words = normalize1(tokens)

    return words

In [48]:
#df_2021_en.info()

In [49]:
# i =0 
# for txt in df_2021_en['text'][:43719].tolist():
#     print(i, replace_contractions(txt))
#     i = i + 1

Create a new column with cleaned text

In [18]:
#df_lock['text_clean_v2'] = df_lock['text'].apply(lambda x: clean_text_v2(x))

In [9]:
df_lock['text_clean_v1'] = df_lock['text'].apply(lambda x: clean_text_v1(x))

In [10]:
df_lock[['text', 'text_clean_v1','text_clean_v2']][:10]

,text,text_clean_v1,text_clean_v2
0,"Win, lose, or tie, Steelers fan till I die 💛🖤🏈...","[win, lose, or, tie, steelers, fan, till, i, d...","[win, lose, tie, steelers, fan, till, die, mon..."
1,"Wind 10.1 mph WNW. Barometer 1004.27 hPa, Risi...","[wind, mph, wnw, barometer, hpa, rising, slowl...","[wind, mph, wnw, barometer, hpa, rising, slowl..."
2,"Wind 7.9 mph W. Barometer 29.95 in, Steady. Te...","[wind, mph, w, barometer, in, steady, temperat...","[wind, mph, w, barometer, steady, temperature,..."
3,#Carpool #Tumwater -&gt; #CrystalMountain ride...,"[carpool, tumwater, gt, crystalmountain, rides...","[carpool, tumwater, gt, crystalmountain, rides..."
4,"22:48 Temp. 10°C, Hum. 83%, Dewp. 6.4°C, Bar. ...","[temp, c, hum, dewp, c, bar, hpa, rain, today,...","[temp, c, hum, dewp, c, bar, hpa, rain, today,..."
5,"0.6 °C, flau vind (0.5 m/s)","[c, flau, vind, ms]","[c, flau, vind, ms]"
6,#Carpool #EastWindsor -&gt; #Windsor rideshare...,"[carpool, eastwindsor, gt, windsor, rideshare,...","[carpool, eastwindsor, gt, windsor, rideshare,..."
7,DOING DOING DOING DOING DOING DOING DOING DOIN...,"[doing, doing, doing, doing, doing, doing, doi...",[uur]
8,ding ding ding ding DONG DONG DONG DONG DONG D...,"[ding, ding, ding, ding, dong, dong, dong, don...","[ding, ding, ding, ding, dong, dong, dong, don..."
9,66.7F (Feels: 63.0F) - Humidity: 56% - Wind: 9...,"[f, feels, f, humidity, wind, mph, ssw, gust, ...","[f, feels, f, humidity, wind, mph, ssw, gust, ..."


Passing through count vectorization

In [12]:
#lstTexts = df_lock['text_clean_v2'].apply(lambda x: " ".join(x)).tolist()
lstTexts

['win lose tie steelers fan till die monaghan sports pub amp grill',
 'wind mph wnw barometer hpa rising slowly temperature c rain today mm humidity',
 'wind mph w barometer steady temperature f rain today humidity',
 'carpool tumwater gt crystalmountain rideshare seat offered splitcosts taketurns',
 'temp c hum dewp c bar hpa rain today mm',
 'c flau vind ms',
 'carpool eastwindsor gt windsor rideshare splitcosts',
 'uur',
 'ding ding ding ding dong dong dong dong dong dong dong dong dong dong dong dong',
 'f feels f humidity wind mph ssw gust mph pressure weather',
 'male dog lost rathkeale road adare lpie',
 'join robert half technology team see latest job openings including c developer via link bio arlington tx',
 'clock allende',
 'ecdbpca',
 'instant pain worth lifetime glories itapema beach',
 'yes got',
 'ever role like buyer like job purchasing mississauga',
 'click link bio see currently open banking jobs like mortg loan originator nmls jr yous bank castlerock co',
 'seas day

In [13]:
import itertools
lstTextToken = df_lock['text_clean_v2'].tolist()
lstTextToken = list(itertools.chain.from_iterable(lstTextToken))
lstTextToken[:1],len(lstTextToken)

(['win'], 713902)

In [14]:
# Keeping the original text
lstTextToken_org = df_lock['text_clean_v1'].tolist()
lstTextToken_org = list(itertools.chain.from_iterable(lstTextToken_org))
lstTextToken_org[:10], len(lstTextToken_org)

(['win',
  'lose',
  'or',
  'tie',
  'steelers',
  'fan',
  'till',
  'i',
  'die',
  'monaghan'],
 1090148)

In [4]:
lstCleanTxt2 = [" ".join(x) for x in df_lock['text_clean_v2'].tolist()]

lstCleanTxt2[:2]


['win lose tie steelers fan till die monaghan sports pub amp grill',
 'wind mph wnw barometer hpa rising slowly temperature c rain today mm humidity']

In [5]:
mkPikle('pickle/2021-4/lstCleanTxt2.pkl', lstCleanTxt2)

In [15]:
# Saving the pickle
mkPikle('pickle/2021-4/lstTextToken.pkl', lstTextToken)
mkPikle('pickle/2021-4/lstTextToken_org.pkl', lstTextToken_org)

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
countVectorizer = CountVectorizer() 
countVector = countVectorizer.fit_transform(lstTexts)

In [23]:
countVector

<76008x87243 sparse matrix of type '<class 'numpy.int64'>'
	with 683467 stored elements in Compressed Sparse Row format>

In [14]:
columns=countVectorizer.get_feature_names()

/usr/local/lib/python3.9/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [15]:
len(columns)

87243

Working with Count vectors

In [17]:
count_vect_df = pd.DataFrame(countVector.toarray(), columns=columns)

In [58]:
#count_vect_df.columns

Saving to pickle

In [11]:
#mkPikle('pickle/2021-4/lstTexts.pkl', lstTexts)
mkPikle('pickle/2021-4/df_lock.pkl', df_lock)
# mkPikle('pickle/2021-4/countVector.pkl', countVector)
# mkPikle('pickle/2021-4/countVectorizer.pkl', countVectorizer)


In [18]:
#mkPikle('pickle/2021-4/count_vect_df.pkl', count_vect_df)

In [23]:
mkPikle('pickle/2021-4/dicCV_lock.pkl', dicCV_lock)
mkPikle('pickle/2021-4/df_lock_summary.pkl', df_lock_summary)

In [17]:
import numpy as np
def getTermValue(wrd):
    lstCV = count_vect_df[wrd].tolist()
    #print(lstCV)
    total_occur = sum(lstCV)
    number_doc  = np.count_nonzero(lstCV)
    return total_occur, number_doc, len(lstCV)
    #return( total_occur * number_doc / len(lstCV))

In [18]:
getTermValue('work')

NameError: name 'count_vect_df' is not defined

In [22]:
import pandas as pd
dicCV_lock = {}
#create dataframe
df_lock_summary = pd.DataFrame({})

for colVal in columns:
    print(colVal) 
    total_occur, number_doc, total = getTermValue(colVal)
    rf = total_occur/ total
    #Add in the dictionary
    dicVal = (total_occur * number_doc) / total
    dicCV_lock[colVal] = dicVal
    # add row
    new_row = {'word':colVal, 'total_occur':total_occur, 'number_docs':number_doc, 'total':total, 
               'rf':rf, 'dicVal': dicVal}
    df_lock_summary = df_lock_summary.append(new_row, ignore_index=True)



1000
12pm
12story
1st
2015
2020
24
412
6p
__
___
____
_____
______
_______
________
_________
__________
___________
____________
_____________
______________
_______________
________________
_________________
__________________
___________________
____________________
_____________________
______________________
_______________________
________________________
_________________________
___________________________
____________________________
_____________________________
______________________________
_______________________________
_________________________________
__________________________________
___________________________________
____________________________________
______________________________________
_______________________________________
________________________________________
__________________________________________
___________________________________________
_____________________________________________
______________________________________________
__________________

adai
adair
adakat
adaku
adalah
adam
adam_oday
adamant
adamawa
adamdare
adamestefani
adamhsays
adamjstephenson
adammacleod
adammcgale
adamrharrison
adams
adamsandsongardens
adamsarette
adamsbeverages
adamshuty
adamspolishes
adamsx_
adamtoison
adamwingard
adanne
adanza
adapt
adaptation
adapted
adapter
adapting
adaption
adaptive
adaptiveresponsive
adare
adarose
adayinthelifeofcarynblack
adazzlessalon
adbcpfef
adcfcafaceddbaddacbafcbdcddeafcafedecbbcabd
adcglobal
add
adda
addam
addamsfamily
addamsfamilything
addance
adde
added
addendum
addesign
addi
addict
addicted
addicted_offic
addicted_under_madonnas_spell
addictedtolove
addictedtoyou
addictforsneakers
addicting
addictio
addiction
addictions
addictive
addicts
addiegail
addilyn
addin
adding
addington
addis
addison
additi
additio
addition
additiona
additional
additions
additives
addlifetoyouryears
addo
address
addressed
addresses
addressing
adds
addy
addyston
ade
adeapfbe
adecco
adega
adegagrill
adekunlegold
adel
adelaide
adelaidebeer
ade

ajman
ajoie
ajongenae
ajperkins
ajsadages
ajstylesbarberandbeautysalon
ajtjewellery
ajxmuxic
ajziggy
ak
aka
akademia
akademiabc
akagery
akai
akaiprofessional
akapeso_
akashacc
akashinga
akbarspeaks
akbash
akc
akcf
akdt
akebono
akeem
akepe
aker
akers
akeyphoto
akf
akf_advocacy
akira
akiraruizphotography
akisnba
aklan
ako
akoko
akon
akoo
akron
aks
aksarbenelmwood
aku
akulaleki
akure
akwa
akwaboahmusic
akylos
akzentz
al
al_be_it
al_connors
al_fliction
ala
alaba
alabama
alabamablissbistro
alabamahills
alabamamikebluescom
alabaster
alabdaly
alacant
aladdin
aladdinsane
aladin
alafiariver
alagbado
alagbara
alai
alaikum
alakaapoor
alakia
alam
alamad
alamance
alamar
alameda
alamedacentral
alamindonesia
alamnusantara
alamo
alamobeerco
alamogordo
alamos
alamosa
alamosaic
alan
alanah
alanparsonsproject
alanpartridge
alanson
alaphia
alariyacomedian
alarm
alarmed
alarmingly
alarms
alas
alaska
alaskaair
alaskaairlines
alaskan
alaskanbrewing
alaskankingsalmon
alaskanmalamute
alaskawildlife
alaskawildl

amapril
amara
amaranth
amares
amaretto
amari
amaricanfootball
amarillo
amarterravillasbalinusad
amaryllis
amash
amaskedlife
amateur
amateurphotography
amateurradio
amateurs
amaurylaporte
amawstudios
amaya
amaz
amaze
amazed
amazine
amazing
amazingamazonians
amazinger
amazingfood
amazingi
amazingly
amazingmy
amazingnaturalhair
amazingshots_moon
amazingsydney
amazingtime
amazo
amazon
amazonas
amazonfba
amazonfresh
amazonia
amazonite
amazonlife
amazonmusic
amazonprime
amazonprimevideo
amazonseller
amba
ambar
ambasaador
ambassador
ambe
amber
amber_bear
amberjack
amberle
ambev
ambiance
ambidextrous
ambience
ambient
ambiepie
ambies
ambitiodesigns
ambition
ambitiospotuv
ambitious
ambivalent
ambivere
ambler
ambleratnight
amblerstrong
amblertheater
ambleside
ambon
ambos
amboy
ambrose
ambrosia
ambrosiac
ambulance
ambulat
ambulatory
amburana
ambury
ambushed
amc
amctalki
amcthewalkingdead
amd
amdg
ame
amelia
amelia_rehberggg
ameliaandadinah
ameliaasimpsonn
ameliak
ameliarose_
ameliaturnbull
amelies

antler
antlers
anto
antofagasta
antoine
antoinesgrill
antoinette
antoinetteconnell
antoinetteelena
anton
antonettas
antoni
antonia
antonio
antoniobayday
antonvenoirinteriorscom
antrim
ants
antsy
antuco
antviv_
antwan
antwerp
antwerpen
anubis
anushka
anvil
anxiety
anxietyproblems
anxietysucks
anxious
anyakubilusphoto
anybody
anycubic_eu
anymo
anymore
anyon
anyone
anyt
anyth
anythin
anything
anytime
anytimefitness
anytimegamer
anytown
anytownusa
anyw
anyway
anywaydo
anyways
anywher
anywhere
anywhereispossible
anywho
anz
anza
anzac
anzaldo
anzworks
ao
aobta
aoc
aod
aoe
aohenry
aonair
aonlineanytime
aooninaustralia
aorus
aotd
aotearoa
aotrembao
aox
ap
ap_lovee
apac
apach
apache
apalit
apart
apartment
apartmentgarden
apartmentlocatingservices
apartments
aparttogether
apav_
ape
apegocrazy
aperitivo
aperol
aperolspritzofficial
aperture
apesce
apex
apexam
apexgaming
apexlegendsindonesia
apexlegendsturkiye
apexmatt_
apexnation
apexpredator
apexusa
apfaa
apha
aphids
aphrodite
api
apicklelypsenow

...

wasabi
wasam
wasamazing
wasatch
wasband
wasdap
waseda
wash
washable
washablerazor
washam
washardcore
washave
washavestl
washbowl
washburn
washburnvalley
washcloths
washed
washer
washerdryer
washes
washi
washin
washing
washingt
washingto
washington
washingtoncrossing
washingtondc
washingtonheights
washingtoniansathome
washingtonsquarepark
washingtonstate
washingtontwp
washoe
washparkdenver
washtenaw
washu_takahashi
washwax
washyadamnhands
washyerhands
washyoass
washyohandspeople
washyourbits
washyourhands
wasilla
wasinger
waskom
wassup
wast
waste
wasted
wastedwednesdays
wasteland
wastemanagement
wastenotwantnot
wastes
wastin
wasting
wastmnk
wat
watabewedding
watabewedding_hawaii
watanabe
watauga
watc
watch
watcha
watchaddict
watchbox
watchcollector
watchcovid
watched
watcher
watchers
watches
watchesofinstagram
watchfam
watchgeek
watchgodwerk
watchin
watching
watchingathome
watchingtangled
watchlilytweet
watchluke
watchmeflyawayu
watchmen
watchmewednesday
watchmixer
watchmyfeet
watchn
wa

wertlieb
werunthestreets
wes
wesbite
wesco
weseeshitdifferent
weshallovercomecorona
weship
weslaco
wesley
wesleyan
wesleymuhammad
wesleysafadao
wesmokemeat
wessex
west
westadams
westaff
westampton
westanna
westauckland
westavl
westbank
westboro
westbound
westbountiful
westbranchstillco
westbrewing
westbrook
westbrookbeer
westbury
westcaldwelltwp
westcentral
westchase
westchester
westchestercountyphotography
westcoast
westcoastswingdance
westcoastteam
westcolumbia
westcorkbrewco
westcovina
westdesmoines
westdyketrail
westend
westendchurchtx
westendmusical
westendorange
wester
westerleigh
westerlind_us
westerly
western
westernillinois
westernmost
westernwings
westeros
westerville
westfield
westfieldlondon
westford
westgate
westgrove
westhampton
westharrison
westheimer
westhighlandterrier
westhighlandwhiteterrier
westhills
westie
westiegram
westies
westiesofinstagram
westilo
westin
westindoha
westisbest
westisland
westislip
westjefferson
westjordan
westkelowna
westlake
westlakevillage
wes

wildecoyete
wilder
wilderness
wildernesscrier
wildernessculture
wildernesssocialdistancing
wildest
wildfire
wildfires
wildflour
wildflower
wildflowers
wildgamecooking
wildhorsegolfclub
wildland
wildleap
wildli
wildlife
wildlifeinthecity
wildlifereserve
wildly
wildlyfeornolyfe
wildman
wildmirto
wildnights
wildoma
wildomar
wildonebirthday
wildoregano
wildridebrewing
wildrosebrewery
wilds
wildside
wildspice
wildthing
wildtwo
wildweatherales
wildwednesday
wildwestminis
wildwood
wildwoods
wiley
wileycollege
wileyfrancomartinez
wilferd
wilford
wilgabah
wilhelm
wilkerson
wilkes
wilkesbarre
wilkins
wilkinsburg
wilkinson
willab
willamette
willcarterband
willcox
willerphotography
willesden
willet
willette
willettized
willfully
willi
william
williamaburgwalk
williamblake
williamblakeartwork
williamblakequote
williamdemeo
williamfm
williamgarder
williamjohnson
williamlos_
williammurraygolf
williamnel
williams
williamsbrewery
williamsburg
williamshakespeare
williamsisland
williamsjrjoe
williamsport

workpushups
workrest
works
worksbell
workselfie
worksh
workshop
workshopping
workshops
worksleeprepeat
worksonpaper
workspace
workstation
workstations
workthanyardwork
worktruck
workwear
workwee
workweek
workweekhustle
workworkwork
worl
worland
worlboss
world
world_isolation
world_wildlife
worldafrodance
worldarchitectureinterior
worldbestgram
worldbodypaintingfestival
worldbookday
worldchampion
worldcircusday
worldclass
worldcrushwednesday
worldearthday
worlderlust
worldfamily
worldfightingchampionships
worldfoodchampionships
worldgonewildmovie
worldhappy
worldhealthday
worldhealthorganisation
worldhealthorganization
worldi
worldofdance
worldofwarcraft
worldpeace
worldpeaceisnationalsecurity
worldpremier
worlds
worldsbestdad
worldshakerstv
worldssznbaby
worldstar
worldstarhiphop
worldstarnews
worldthis
worldtraveler
worldtravler
worldunion
worldunited
worldwi
worldwide
worldwidefm
worldwideshipping
worldwidesolidarity
worldwidewoowoo
worm
worms
wormtail
worn
worr
worried
worrier
worri

yogurt_daddy
yogurts
yogya
yogyakarta
yohanesyoedha
yoint
yointcounty
yoke
yoko
yokohama
yokosonews
yola
yolanda
yolandaadams
yolatengo
yolk
yolked
yolks
yolo
yolunchbox
yom
yomama
yomequedoencasa
yomtvraps
yon
yona
yonder
yoneya
yong
yonge
yongedundas
yoni
yonisteams
yonkers
yonkersny
yonko
yonkotv
yonks
yoo
yoohoo
yooo
yooohooo
yoooo
yooooo
yooooooou
yor
yorbalinda
york
yorkcopa
yorke
yorker
yorkers
yorkiegram
yorkiepoo
yorkiepoosofinstagram
yorkiepup
yorkies
yorkshire
yorkshirepudding
yorkshireterrier
yorkton
yorkville
yoruba
yorunner
yosemite
yosemitenationalpark
yosemitevalley
yoshi
yoshinocherry
yoshinomitsu
yosimitesam
yosoyma
yostonyos
yosua
yotamafia
yotei
you_mus_cum
youare
youareloved
youarenotalone
youarestrong
youarewearingthesamethingtoday
youbready
youbut
youcandancedallas
youcandancewithus
youcandoit
youcanfeelitinthestreets
youcanhelp
youcanneverhavetoomanycookbooks
youcanstillrockinamerica
youcanttellmenothing
youcollme
youd
youdontknowbo
youdontknowmystory
youdoyou
y

In [28]:
sorted(dicCV_lock.items(), key=lambda kv: kv[1],reverse=True)

[('job', 326.13380170508367),
 ('new', 240.25694663719608),
 ('like', 195.98489632670245),
 ('day', 172.02644458478056),
 ('today', 124.91806125670982),
 ('amp', 114.59985790969371),
 ('link', 101.20382065045784),
 ('us', 100.2414482686033),
 ('home', 97.23525155246816),
 ('see', 96.76349857909693),
 ('posted', 92.25218398063362),
 ('click', 89.89832649194821),
 ('bio', 84.84712135564678),
 ('photo', 83.48259393748026),
 ('time', 79.65236554046942),
 ('pm', 72.85588359120092),
 ('drinking', 69.29482422902852),
 ('happy', 62.248631722976526),
 ('one', 55.30438901168298),
 ('love', 54.4785680454689),
 ('great', 47.26476160404168),
 ('apply', 46.931691400905166),
 ('work', 45.904838964319545),
 ('latest', 43.19734764761604),
 ('ca', 40.22000315756236),
 ('get', 37.18703294390064),
 ('hiring', 34.59191137774971),
 ('good', 32.15227344490054),
 ('th', 31.6277233975371),
 ('california', 30.356344069045363),
 ('want', 29.738395958320176),
 ('quarantine', 28.246868750657825),
 ('looking', 27.9

In [29]:
lst_100 = sorted(dicCV_lock.items(), key=lambda kv: kv[1],reverse=True)[:100]
# Creating a top 100 set of words 
top_100 = list(map(lambda x: x[0], lst_100))
top_100[:5]

['job', 'new', 'like', 'day', 'today']

In [30]:
mkPikle('pickle/2021-4/top_100.pkl', top_100)

In [25]:
df_lock_summary[(df_lock_summary.total_occur>2) & (df_lock_summary.number_docs>3) 
                & (df_lock_summary.word.str.len()>2) & (df_lock_summary.dicVal>1)][:2]

,word,total_occur,number_docs,total,rf,dicVal
1331,ago,410.0,408.0,76008.0,0.005394,2.200821
1466,air,543.0,541.0,76008.0,0.007144,3.864896


In [26]:
df_lock_summary[(df_lock_summary.total_occur>2) & (df_lock_summary.number_docs>3) 
                & (df_lock_summary.word.str.len()>2) & (df_lock_summary.dicVal>1)].count()

word           349
total_occur    349
number_docs    349
total          349
rf             349
dicVal         349
dtype: int64

In [6]:
df_lock_summary[(df_lock_summary.total_occur>2) & (df_lock_summary.number_docs>3) 
                 & (df_lock_summary.dicVal>1)].sort_values(by=['total_occur'], ascending=False)[:50]

,word,total_occur,number_docs,total,rf,dicVal
39650,job,5061.0,4898.0,76008.0,0.066585,326.133802
52621,new,4509.0,4050.0,76008.0,0.059323,240.256947
44148,like,3983.0,3740.0,76008.0,0.052402,195.984896
19107,day,3722.0,3513.0,76008.0,0.048969,172.026445
78558,today,3111.0,3052.0,76008.0,0.040930,124.918061
2560,amp,3066.0,2841.0,76008.0,0.040338,114.599858
81410,us,2824.0,2698.0,76008.0,0.037154,100.241448
44362,link,2775.0,2772.0,76008.0,0.036509,101.203821
34940,home,2767.0,2671.0,76008.0,0.036404,97.235252
67965,see,2724.0,2700.0,76008.0,0.035838,96.763499


In [19]:
df_lock_summary[df_lock_summary.word.isin(
    ['covid','virus','quarantine','lockdown','mask', 'distance','corona'])]

,word,total_occur,number_docs,total,rf,dicVal
16685,corona,228.0,225.0,76008.0,0.003000,0.674929
17112,covid,1185.0,1138.0,76008.0,0.015590,17.741948
20859,distance,128.0,128.0,76008.0,0.001684,0.215556
44846,lockdown,411.0,393.0,76008.0,0.005407,2.125079
47386,mask,324.0,305.0,76008.0,0.004263,1.300126
61732,quarantine,1492.0,1439.0,76008.0,0.019630,28.246869
82556,virus,132.0,131.0,76008.0,0.001737,0.227502


In [4]:
df_lock_summary[df_lock_summary.word.isin(
    ['covid'])]

,word,total_occur,number_docs,total,rf,dicVal
17112,covid,1185.0,1138.0,76008.0,0.01559,17.741948


In [23]:
import nltk
from nltk.collocations import * 

def getColocation(text, cls, typeText):
    if typeText=="org":
        lstText = lstTextToken_org
    else:
        lstText = lstTextToken
    # Ngrams with 'creature' as a member
    bigram_measures = nltk.collocations.BigramAssocMeasures()
    trigram_measures = nltk.collocations.TrigramAssocMeasures()# Ngrams with 'creature' as a member
    
    ## Bigrams
    finder2 = BigramCollocationFinder.from_words(lstText)
    finder3 = TrigramCollocationFinder.from_words(lstTextToken)


    if cls==2:
        cls = bigram_measures
        finder = finder2
    else:
        cls = trigram_measures
        finder = finder3
        
    creature_filter = lambda *w: text not in w

    # only bigrams that appear 3+ times
    #finder.apply_freq_filter(2)

    # only bigrams that contain 'creature'
    finder.apply_ngram_filter(creature_filter)

    # return the 10 n-grams with the highest PMI
    print (finder.nbest(cls.likelihood_ratio, 50))
    
    count = 0 
    
    for i in finder.score_ngrams(cls.likelihood_ratio):
        if count ==50:
            break;
        print (i) 
        count = count + 1

In [34]:
getColocation("quarantine",2,"org")

[('during', 'quarantine'), ('quarantine', 'day'), ('quarantine', 'quarantinelife'), ('this', 'quarantine'), ('quarantine', 'stayhome'), ('of', 'quarantine'), ('self', 'quarantine'), ('quarantine', 'has'), ('quarantine', 'life'), ('in', 'quarantine'), ('quarantinelife', 'quarantine'), ('quarantine', 'carneasada'), ('covid', 'quarantine'), ('under', 'quarantine'), ('quarantine', 'got'), ('stayhome', 'quarantine'), ('the', 'quarantine'), ('quarantine', 'buddy'), ('quarantine', 'jeanferreirajf'), ('quarantine', 'cooking'), ('i', 'quarantine'), ('quarantine', 'edition'), ('quarantine', 'a'), ('quarantine', 'workfromhome'), ('my', 'quarantine'), ('quarantine', 'activity'), ('quarantine', 'the'), ('quarantine', 'is'), ('dinnerathome', 'quarantine'), ('quarantine', 'thegreattpcrisisof'), ('quarantinequote', 'quarantine'), ('pandemic', 'quarantine'), ('quarantine', 'to'), ('quarantine', 'blues'), ('quarantine', 'mood'), ('quarantine', 'cuisine'), ('quarantine', 'easter'), ('quarantine', 'dinner

In [35]:
getColocation("quarantine",3,"org")

[('link', 'bio', 'quarantine'), ('quarantine', 'link', 'bio'), ('posted', 'photo', 'quarantine'), ('quarantine', 'new', 'york'), ('new', 'york', 'quarantine'), ('untappd', 'home', 'quarantine'), ('quarantine', 'los', 'angeles'), ('quarantine', 'see', 'latest'), ('quarantine', 'recommend', 'anyone'), ('endomondo', 'endorphins', 'quarantine'), ('course', 'groundspeed', 'quarantine'), ('pm', 'cdt', 'quarantine'), ('quarantine', 'want', 'work'), ('quarantine', 'san', 'francisco'), ('san', 'francisco', 'quarantine'), ('quarantine', 'severe', 'thunderstorm'), ('foodwaste', 'free', 'quarantine'), ('quarantine', 'let', 'us'), ('quarantine', 'social', 'distancing'), ('social', 'distancing', 'quarantine'), ('pressure', 'mb', 'quarantine'), ('utc', 'quake', 'quarantine'), ('happy', 'easter', 'quarantine'), ('quarantine', 'happy', 'easter'), ('groundspeed', 'kts', 'quarantine'), ('oh', 'realestate', 'quarantine'), ('new', 'jersey', 'quarantine'), ('quarantine', 'z', 'npd'), ('north', 'carolina', '

<H1> Creating the Word embedding from the sentences</H1>

In [12]:
from gensim.models import Word2Vec
# define training data
sentences = df_lock['text_clean_v1'].tolist()
# train model
model = Word2Vec(sentences, min_count=1)
# summarize the loaded model
print(model)
# summarize vocabulary
words = list(model.wv.vocab)
#print(words)
# access vector for one word
#print(model['sentence'])
# save model
model.save('pickle/2021-4/model.bin')

Word2Vec(vocab=87968, size=100, alpha=0.025)


In [13]:
model.most_similar("stay",  topn=20)

/tmp/ipykernel_15772/1736541646.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar("stay",  topn=20)


[('staying', 0.911885142326355),
 ('healthy', 0.8978226184844971),
 ('safe', 0.8843215703964233),
 ('stuck', 0.8531713485717773),
 ('mwa', 0.8448354005813599),
 ('dud', 0.8273288607597351),
 ('positive', 0.826276421546936),
 ('tuned', 0.8162994384765625),
 ('motivated', 0.8146123886108398),
 ('together', 0.8144313097000122),
 ('strong', 0.8106278777122498),
 ('keep', 0.8065625429153442),
 ('najwasgarden', 0.805801510810852),
 ('keeping', 0.795488715171814),
 ('growbeard', 0.794830322265625),
 ('whatisimportant', 0.791567862033844),
 ('vigilant', 0.7903071045875549),
 ('solutionsfocused', 0.7872575521469116),
 ('paliecmjs', 0.7869130373001099),
 ('wapato', 0.7857635021209717)]

In [14]:
model.most_similar("stayathome",  topn=20)

/tmp/ipykernel_15772/3787337356.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar("stayathome",  topn=20)


[('pls', 0.9339054822921753),
 ('counts', 0.9296010136604309),
 ('socialdistancing', 0.9131410717964172),
 ('chaotic', 0.9093873500823975),
 ('stayhomenz', 0.9087285995483398),
 ('usedcarsforsale', 0.9068558216094971),
 ('chronicles', 0.9047456383705139),
 ('iraq', 0.9008088111877441),
 ('wfh', 0.8982254266738892),
 ('stayhome', 0.8974696397781372),
 ('denmark', 0.895855724811554),
 ('herd', 0.8952124118804932),
 ('aging', 0.893872857093811),
 ('tgif', 0.892949104309082),
 ('testing', 0.8929400444030762),
 ('quarantinebutmakeitfashion', 0.8890634775161743),
 ('complete', 0.8872503638267517),
 ('quarentine', 0.8864750862121582),
 ('staysafe', 0.885634183883667),
 ('yashicamm', 0.8853801488876343)]

In [5]:
model.most_similar("distance",  topn=20)

/tmp/ipykernel_30797/237723630.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar("distance",  topn=20)


[('distancing', 0.9430010914802551),
 ('practicing', 0.907896101474762),
 ('holi', 0.8350707292556763),
 ('self', 0.8107497692108154),
 ('practice', 0.8094589710235596),
 ('distancin', 0.7998688220977783),
 ('donation', 0.7965419888496399),
 ('festivalofcolors', 0.7925224900245667),
 ('markers', 0.7831828594207764),
 ('distanc', 0.7824870944023132),
 ('isolation', 0.7789778113365173),
 ('maintaining', 0.777380645275116),
 ('itsnotwinter', 0.7748892307281494),
 ('legs', 0.7748717069625854),
 ('maintain', 0.772681713104248),
 ('social', 0.7715981006622314),
 ('calm', 0.7703343629837036),
 ('body', 0.7698159217834473),
 ('quarantined', 0.7696516513824463),
 ('suffering', 0.7694437503814697)]

In [6]:
model.most_similar("corona",  topn=20)

/tmp/ipykernel_30797/2686852255.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar("corona",  topn=20)


[('virus', 0.9311869144439697),
 ('isolation', 0.9187617301940918),
 ('socialdistancing', 0.9007528424263),
 ('mlscovid', 0.8897266387939453),
 ('quarantining', 0.8888655304908752),
 ('amidst', 0.8854901194572449),
 ('forced', 0.8775936961174011),
 ('selfisolation', 0.8773389458656311),
 ('under', 0.876122236251831),
 ('happiness', 0.8748174905776978),
 ('confinement', 0.8745551109313965),
 ('pandemic', 0.8744563460350037),
 ('beat', 0.8726783990859985),
 ('sanitize', 0.8721603155136108),
 ('nature', 0.8708432912826538),
 ('vibes', 0.8705838322639465),
 ('complete', 0.8701725602149963),
 ('quarantinelife', 0.8688712120056152),
 ('breathing', 0.868161141872406),
 ('stayhome', 0.8679975271224976)]

In [7]:
model.most_similar("covid",  topn=20)

/tmp/ipykernel_30797/2724363481.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar("covid",  topn=20)


[('coronavirus', 0.9519915580749512),
 ('pandemic', 0.8882442712783813),
 ('lockdown', 0.8513996601104736),
 ('corona', 0.8352657556533813),
 ('outbreak', 0.8263430595397949),
 ('virus', 0.8237357139587402),
 ('stayhome', 0.806885302066803),
 ('insideabc', 0.8050203323364258),
 ('outsideabc', 0.8020122051239014),
 ('fighting', 0.7975575923919678),
 ('mlscovid', 0.790101170539856),
 ('crisis', 0.7817627191543579),
 ('socialdistancing', 0.7808501720428467),
 ('spread', 0.7777633666992188),
 ('dothis', 0.7765145301818848),
 ('isolation', 0.7764136791229248),
 ('novisitorsallowed', 0.7743579149246216),
 ('empty', 0.7737635374069214),
 ('yashicamm', 0.7735843658447266),
 ('amidst', 0.77070552110672)]

In [8]:
model.most_similar("lockdown",  topn=20)

/tmp/ipykernel_30797/523177419.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar("lockdown",  topn=20)


[('isolation', 0.9145526885986328),
 ('pandemic', 0.8583959341049194),
 ('corona', 0.8581873774528503),
 ('socialdistancing', 0.8550847768783569),
 ('covid', 0.8513997197151184),
 ('crisis', 0.8506446480751038),
 ('confinement', 0.8497622013092041),
 ('coronavirus', 0.8491284847259521),
 ('virus', 0.8391594886779785),
 ('nature', 0.8315749764442444),
 ('flowers', 0.8313511610031128),
 ('stayathome', 0.8272786140441895),
 ('complete', 0.8250555992126465),
 ('notes', 0.8246129155158997),
 ('mlscovid', 0.8210247755050659),
 ('number', 0.8185747265815735),
 ('testing', 0.8176553249359131),
 ('cmfl', 0.8173964619636536),
 ('quarantine', 0.8172634840011597),
 ('rate', 0.8112494945526123)]

In [9]:
model.most_similar("mask",  topn=20)

/tmp/ipykernel_30797/3037489044.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar("mask",  topn=20)


[('face', 0.9279518723487854),
 ('hair', 0.8975942134857178),
 ('body', 0.8636550903320312),
 ('kids', 0.8416284918785095),
 ('girl', 0.8414108753204346),
 ('smile', 0.8407293558120728),
 ('without', 0.8392840623855591),
 ('makeup', 0.8320345878601074),
 ('whole', 0.8319076299667358),
 ('wearing', 0.83022141456604),
 ('baby', 0.828359842300415),
 ('money', 0.8274693489074707),
 ('into', 0.827007532119751),
 ('self', 0.8222320079803467),
 ('eyes', 0.82209312915802),
 ('simple', 0.8218981027603149),
 ('content', 0.8218823671340942),
 ('uplifting', 0.8217043876647949),
 ('masks', 0.8210534453392029),
 ('husband', 0.8193084597587585)]

In [10]:
model.most_similar("quarantine",  topn=20)

/tmp/ipykernel_30797/2366894413.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar("quarantine",  topn=20)


[('isolation', 0.8401468992233276),
 ('pandemic', 0.8297849893569946),
 ('game', 0.8276929259300232),
 ('whole', 0.8268992900848389),
 ('life', 0.8195234537124634),
 ('super', 0.817357063293457),
 ('lockdown', 0.8172634840011597),
 ('nature', 0.8103944063186646),
 ('corona', 0.8031185865402222),
 ('heart', 0.7858638167381287),
 ('other', 0.7848529815673828),
 ('taking', 0.7814793586730957),
 ('making', 0.7779214382171631),
 ('crazy', 0.7766737937927246),
 ('lost', 0.7755255699157715),
 ('hubs', 0.7748629450798035),
 ('playing', 0.7747044563293457),
 ('superbowl', 0.7743562459945679),
 ('baby', 0.7737784385681152),
 ('rough', 0.7728859782218933)]

In [15]:
model.most_similar("virus",  topn=20)

/tmp/ipykernel_15772/1525382260.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar("virus",  topn=20)


[('pandemic', 0.9179227948188782),
 ('corona', 0.9089281558990479),
 ('aprilst', 0.872631847858429),
 ('period', 0.8641219139099121),
 ('chaotic', 0.8567099571228027),
 ('lockdown', 0.8549883365631104),
 ('spread', 0.8548483848571777),
 ('crisis', 0.8546954393386841),
 ('isolation', 0.8469161987304688),
 ('covid', 0.8460092544555664),
 ('fight', 0.8437996506690979),
 ('against', 0.8425061702728271),
 ('under', 0.8420249819755554),
 ('clearly', 0.8412595987319946),
 ('coronavirus', 0.8402547836303711),
 ('changed', 0.8390491604804993),
 ('season', 0.834962010383606),
 ('words', 0.832650899887085),
 ('complete', 0.831954836845398),
 ('fear', 0.8312507271766663)]

<h2> Topic modeling</h2>

In [20]:
from bertopic import BERTopic
modelT = BERTopic(min_topic_size=20)
topics, probabilities = modelT.fit_transform(lstCleanTxt2)

In [21]:
# 2 times reduction
#topics, probabilities = modelT.reduce_topics(lstCleanTxt2, topics, probabilities, nr_topics="auto")
#topics, probabilities = modelT.reduce_topics(lstCleanTxt2, topics, probabilities, nr_topics="auto")

In [23]:
# Saving the model
modelT.save("pickle/2021-4/modelBertTopic")

/usr/local/lib/python3.9/dist-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [24]:
dfTp= modelT.get_topic_info()
dfTp

,Topic,Count,Name
0,-1,28649,-1_click_see_day_love
1,0,3438,0_drinking_untappd_beer_home
2,1,3058,1_jobs_job_titles_follow
3,2,2147,2_music_live_song_tune
4,3,1728,3_easter_birthday_happy_bunny
...,...,...,...
217,216,22,216_shelter_place_sheltering_shelterinplace
218,217,21,217_gas_fuel_gallon_prices
219,218,21,218_skating_skate_skates_skaters
220,219,21,219_nubias_thru_drive_asada


In [92]:
dfTp['TopicWords']= dfTp.apply (lambda row: dicTopWords[row.Topic], axis=1)

In [93]:
dfTp[:21]

,Topic,Count,Name,TopicWords
0,-1,28649,-1_click_see_day_love,"[click, see, day, love, link, california, job, today, latest, time]"
1,0,3438,0_drinking_untappd_beer_home,"[drinking, untappd, beer, home, ipa, brewing, ale, wine, stout, drink]"
2,1,3058,1_jobs_job_titles_follow,"[jobs, job, titles, follow, career, hear, like, sales, appealing, fit]"
3,2,2147,2_music_live_song_tune,"[music, live, song, tune, album, spotify, dj, podcast, soundcloud, tonight]"
4,3,1728,3_easter_birthday_happy_bunny,"[easter, birthday, happy, bunny, family, happyeaster, everyone, eggs, sunday, hope]"
5,4,1394,4_pizza_dinner_chicken_cake,"[pizza, dinner, chicken, cake, garlic, cookies, sauce, pork, rice, fried]"
6,5,1241,5_quarantine_quarantinelife_quarantined_day,"[quarantine, quarantinelife, quarantined, day, stayhome, life, quarantinecooking, cooking, best, quarantineandchill]"
7,6,1237,6_church_god_jesus_life,"[church, god, jesus, life, bible, christ, prayer, lord, faith, pastor]"
8,7,969,7_texas_houston_chicago_tennessee,"[texas, houston, chicago, tennessee, posted, ohio, photo, illinois, missouri, kansas]"
9,8,787,8_york_bronx_new_boston,"[york, bronx, new, boston, brooklyn, massachusetts, street, manhattan, photo, posted]"


In [25]:
# Create a dictionary and set of words
dicTopWords = {}
setTopWords = set()

for topic in dfTp['Topic'].tolist():
    lstA               = [a for (a,b) in modelT.get_topic(topic)]
    dicTopWords[topic] = lstA # dic
    setTopWords.update(lstA)  # set

In [94]:
mkPikle('pickle/2021-4/dicTopWords.pkl', dicTopWords)
mkPikle('pickle/2021-4/setTopWords.pkl', setTopWords)
mkPikle('pickle/2021-4/dfTp.pkl', dfTp)
mkPikle('pickle/2021-4/topics.pkl', topics)
mkPikle('pickle/2021-4/probabilities.pkl', probabilities)

In [95]:
len(setTopWords)

1904

In [ ]:
# finding covid related word in dictionary


In [80]:
# mystring = "3:mas1"
# if "mask" in mystring: 
#    print('success')
def getTopicsByWord(chkWord):
    for key in dicTopWords:
        #print(key)
        for word in dicTopWords[key]:
            if chkWord in word:
                print(key, dfTp[dfTp.Topic==key]['Name'].tolist()[0], dicTopWords[key], 
                      dfTp[dfTp.Topic==key]['Count'].tolist()[0])
                break

In [88]:
getTopicsByWord("stay")

5 5_quarantine_quarantinelife_quarantined_day ['quarantine', 'quarantinelife', 'quarantined', 'day', 'stayhome', 'life', 'quarantinecooking', 'cooking', 'best', 'quarantineandchill'] 1241
24 24_coronavirus_covid_virus_pandemic ['coronavirus', 'covid', 'virus', 'pandemic', 'corona', 'lockdown', 'stayhome', 'update', 'tests', 'outbreak'] 543
33 33_amp_safe_call_stay ['amp', 'safe', 'call', 'stay', 'got', 'back', 'keep', 'need', 'many', 'things'] 393
145 145_staysafe_weloveourhealthcareworkers_stayhome_stayhealthy ['staysafe', 'weloveourhealthcareworkers', 'stayhome', 'stayhealthy', 'greenwich', 'stay', 'gratitude', 'village', 'staystrong', 'healthy'] 45


20